In [134]:
source("../00_model-fitting/wwa_model-fitting.R")
library(extRemes)
library(plyr)

In [42]:
# load data
gmst <- read.fwf("../../Data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
nino <- read.csv("data/nino34rel_september.csv")
iod <- read.csv("data/iod_september.csv")

---
## **Check significance of other covariates (normal linear model)**

In [93]:
# load data
fnm <- "data/era5_madagascar_t2m-mean.csv"
df <- merge(merge(merge(gmst, nino), iod), read.csv(fnm))

mdls <- c("gmst", "nino34", "iod", "gmst + nino34", "gmst * nino34", "gmst + iod", "gmst * iod")
res <- rbind.fill(sapply(mdls, function(f) data.frame(t(summary(lm(paste0("var ~ ", f), df))$coef[,4])), simplify = F)) < 0.05
rownames(res) <- mdls
res

,X.Intercept.,gmst,nino34,iod,gmst.nino34,gmst.iod
gmst,TRUE,TRUE,NA,NA,NA,NA
nino34,TRUE,NA,FALSE,NA,NA,NA
iod,TRUE,NA,NA,TRUE,NA,NA
gmst + nino34,TRUE,TRUE,FALSE,NA,NA,NA
gmst * nino34,TRUE,TRUE,FALSE,NA,FALSE,NA
gmst + iod,TRUE,TRUE,NA,FALSE,NA,NA
gmst * iod,TRUE,TRUE,NA,FALSE,NA,FALSE


In [94]:
# load data
fnm <- "data/era5_antananarivo_tx7x.csv"
df <- merge(merge(merge(gmst, nino), iod), read.csv(fnm))

mdls <- c("gmst", "nino34", "iod", "gmst + nino34", "gmst * nino34", "gmst + iod", "gmst * iod")
res <- rbind.fill(sapply(mdls, function(f) data.frame(t(summary(lm(paste0("var ~ ", f), df))$coef[,4])), simplify = F)) < 0.05
rownames(res) <- mdls
res

,X.Intercept.,gmst,nino34,iod,gmst.nino34,gmst.iod
gmst,TRUE,TRUE,NA,NA,NA,NA
nino34,TRUE,NA,FALSE,NA,NA,NA
iod,TRUE,NA,NA,FALSE,NA,NA
gmst + nino34,TRUE,TRUE,FALSE,NA,NA,NA
gmst * nino34,TRUE,TRUE,FALSE,NA,FALSE,NA
gmst + iod,TRUE,TRUE,NA,FALSE,NA,NA
gmst * iod,TRUE,TRUE,NA,FALSE,NA,FALSE


In [95]:
# load data
fnm <- "data/era5_antananarivo_tn7x.csv"
df <- merge(merge(merge(gmst, nino), iod), read.csv(fnm))

mdls <- c("gmst", "nino34", "iod", "gmst + nino34", "gmst * nino34", "gmst + iod", "gmst * iod")
res <- rbind.fill(sapply(mdls, function(f) data.frame(t(summary(lm(paste0("var ~ ", f), df))$coef[,4])), simplify = F)) < 0.05
rownames(res) <- mdls
res

,X.Intercept.,gmst,nino34,iod,gmst.nino34,gmst.iod
gmst,TRUE,TRUE,NA,NA,NA,NA
nino34,TRUE,NA,FALSE,NA,NA,NA
iod,TRUE,NA,NA,TRUE,NA,NA
gmst + nino34,TRUE,TRUE,FALSE,NA,NA,NA
gmst * nino34,TRUE,TRUE,FALSE,NA,FALSE,NA
gmst + iod,TRUE,TRUE,NA,TRUE,NA,NA
gmst * iod,TRUE,TRUE,NA,FALSE,NA,FALSE


In [132]:
summary(lm(var ~ gmst + iod, df))


Call:
lm(formula = var ~ gmst + iod, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.88105 -0.55322 -0.02208  0.56428  1.85956 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  13.4017     0.1173 114.249  < 2e-16 ***
gmst          1.4224     0.2638   5.392 8.68e-07 ***
iod           0.3271     0.1586   2.062   0.0428 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7371 on 71 degrees of freedom
Multiple R-squared:  0.3614,	Adjusted R-squared:  0.3434 
F-statistic: 20.09 on 2 and 71 DF,  p-value: 1.218e-07


---
## **LR test with GEV**

In [102]:
fnm <- "data/era5_antananarivo_tn7x.csv"
df <- merge(merge(merge(gmst, nino), iod), read.csv(fnm))

In [128]:
mdl_gmst <- fevd(var, df, location.fun = ~ gmst)
mdl_gmst_iod <- fevd(var, df, location.fun = ~ gmst + iod)

In [133]:
c(summary(mdl_gmst)$AIC, summary(mdl_gmst_iod)$AIC)


fevd(x = var, data = df, location.fun = ~gmst)

[1] "Estimation Method used: MLE"


 Negative Log-Likelihood Value:  82.63551 


 Estimated parameters:
       mu0        mu1      scale      shape 
13.1760812  1.4863778  0.7475926 -0.3006160 

 Standard Error Estimates:
       mu0        mu1      scale      shape 
0.12769907 0.25479311 0.06735000 0.07395368 

 Estimated parameter covariance matrix.
                mu0          mu1         scale        shape
mu0    0.0163070536 -0.021670554  0.0002946562 -0.004054607
mu1   -0.0216705540  0.064919527 -0.0012506890  0.004498019
scale  0.0002946562 -0.001250689  0.0045360218 -0.002859641
shape -0.0040546073  0.004498019 -0.0028596410  0.005469146

 AIC = 173.271 

 BIC = 182.4873 

fevd(x = var, data = df, location.fun = ~gmst + iod)

[1] "Estimation Method used: MLE"


 Negative Log-Likelihood Value:  80.59427 


 Estimated parameters:
       mu0        mu1        mu2      scale      shape 
13.1633367  1.3615900  0.3199600  0.7171638 -0.2

[1] 173.2710 171.1885